<a href="https://colab.research.google.com/github/BenzterBit/MLP/blob/master/AnimeRecommendationSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"benzybit","key":"2d9bccaa7d4d615e650a0b725f35bb7e"}'}

In [0]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [65]:
!kaggle datasets download -d CooperUnion/anime-recommendations-database -p/content
!unzip \*.zip

anime-recommendations-database.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  anime-recommendations-database.zip
replace anime.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: anime.csv               
  inflating: rating.csv              


In [39]:

import pandas as pd
import numpy as np
anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')
#rating_df.head(20)
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [40]:
anime_df['genre'] = anime_df.genre.str.split(',')
anime_df = anime_df.drop(['type','rating','episodes','members'],axis=1)
anime_df = anime_df.dropna()
anime_df.head()

,anime_id,name,genre
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]"
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,..."
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur..."
3,9253,Steins;Gate,"[Sci-Fi, Thriller]"
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur..."


In [41]:
anime_genre_df = anime_df.copy()

for index, row in anime_df.iterrows():
  for gen in row['genre']:
    anime_genre_df.at[index,gen]=1
anime_genre_df = anime_genre_df.fillna(0)
anime_genre_df.head()

,anime_id,name,genre,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,28977,Gintama°,"[Action, Comedy, Historical, Parody, Samur...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9969,Gintama&#039;,"[Action, Comedy, Historical, Parody, Samur...",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
rating_df.head()
rating_df.shape
msk = np.random.rand(len(rating_df)) <0.02

ratingless_df = rating_df[msk]
ratingless_df.shape

(156494, 3)

In [0]:
for index, row in ratingless_df.iterrows():
  if row['rating']==-1:
    ratingless_df =ratingless_df.drop([index], axis = 0)
  

In [44]:
ratingless_df.shape

(126611, 3)

In [45]:
ratingless_df.head()

,user_id,anime_id,rating
182,3,4026,7
331,5,199,8
409,5,1956,2
422,5,2418,9
444,5,4015,1


In [58]:
userInput = [
            {'name':'Kimi no Na wa.', 'rating':9},
            {'name':'Fullmetal Alchemist: Brotherhood', 'rating':8.5},
            {'name':'Steins;Gate', 'rating':9.5},
            {'name':"Death Note", 'rating':10},
            {'name':'Mob Psycho 100', 'rating':8.5}
         ] 
input_movies = pd.DataFrame(userInput)


input_movies.head()

,name,rating
0,Kimi no Na wa.,9.0
1,Fullmetal Alchemist: Brotherhood,8.5
2,Steins;Gate,9.5
3,Death Note,10.0
4,Mob Psycho 100,8.5


In [62]:
input_id = anime_df[anime_df['name'].isin(input_movies['name'])]
input_id = input_id.drop('genre',1)
input_movies = pd.merge(input_id,input_movies)
input_movies

,anime_id,name,rating
0,32281,Kimi no Na wa.,9.0
1,5114,Fullmetal Alchemist: Brotherhood,8.5
2,9253,Steins;Gate,9.5
3,1535,Death Note,10.0
4,32182,Mob Psycho 100,8.5


In [68]:
user_anime = anime_genre_df[anime_genre_df['anime_id'].isin(input_movies['anime_id'])]
user_anime

,anime_id,name,genre,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,32281,Kimi no Na wa.,"[Drama, Romance, School, Supernatural]",1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5114,Fullmetal Alchemist: Brotherhood,"[Action, Adventure, Drama, Fantasy, Magic,...",0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9253,Steins;Gate,"[Sci-Fi, Thriller]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
40,1535,Death Note,"[Mystery, Police, Psychological, Supernatur...",0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
82,32182,Mob Psycho 100,"[Action, Comedy, Slice of Life, Supernatural]",0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [69]:
user_anime = user_anime.reset_index(drop=True)
user_genre_table =user_anime.drop(['anime_id','name','genre'],1)
user_genre_table

,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,Demons,Ecchi,Josei,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [70]:
user_profile = user_genre_table.T.dot(input_movies['rating'])
user_profile

Drama              9.0
 Romance           9.0
 School            9.0
 Supernatural     27.5
Action            17.0
 Adventure         8.5
 Drama             8.5
 Fantasy           8.5
 Magic             8.5
 Military          8.5
 Shounen           8.5
 Comedy            8.5
 Historical        0.0
 Parody            0.0
 Samurai           0.0
 Sci-Fi            0.0
Sci-Fi             9.5
 Thriller         19.5
Comedy             0.0
 Sports            0.0
 Super Power       0.0
 Space             0.0
 Slice of Life     8.5
 Mecha             0.0
Adventure          0.0
 Music             0.0
 Mystery           0.0
 Seinen            0.0
Fantasy            0.0
 Martial Arts      0.0
                  ... 
 Dementia          0.0
Cars               0.0
Mecha              0.0
Horror             0.0
School             0.0
 Kids              0.0
Historical         0.0
Kids               0.0
 Shoujo Ai         0.0
Shounen            0.0
Shoujo             0.0
Magic              0.0
Harem      

In [86]:
genre_table = anime_genre_df.set_index(anime_genre_df['anime_id'])
genre_table = genre_table.drop(['anime_id','name','genre'],1)
genre_table.head()

,Drama,Romance,School,Supernatural,Action,Adventure,Drama,Fantasy,Magic,Military,Shounen,Comedy,Historical,Parody,Samurai,Sci-Fi,Sci-Fi,Thriller,Comedy,Sports,Super Power,Space,Slice of Life,Mecha,Adventure,Music,Mystery,Seinen,Fantasy,Martial Arts,Vampire,Shoujo,Horror,Police,Psychological,Mystery,Psychological,Demons,Ecchi,Josei,...,Romance,Shounen Ai,Game,Demons,Ecchi,Dementia,Harem,Music,Game,Cars,Dementia,Cars,Mecha,Horror,School,Kids,Historical,Kids,Shoujo Ai,Shounen,Shoujo,Magic,Harem,Martial Arts,Sports,Slice of Life,Seinen,Parody,Police,Thriller,Supernatural,Samurai,Super Power,Vampire,Space,Hentai,Yaoi,Hentai,Yuri,Yaoi
anime_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
32281,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5114,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28977,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9253,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9969,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
recom_table_df = ((genre_table*user_profile).sum(axis=1)/user_profile.sum())
recom_table_df.head(20)

anime_id
32281    0.274559
5114     0.342569
28977    0.171285
9253     0.146096
9969     0.171285
32935    0.130982
11061    0.171285
820      0.088161
15335    0.171285
15417    0.171285
4181     0.314861
28851    0.133501
918      0.171285
2904     0.171285
28891    0.130982
199      0.181360
23273    0.178841
24701    0.224181
12355    0.042821
1575     0.173804
dtype: float64

In [83]:
recom_table_sort_df = recom_table_df.sort_values(ascending=False)
recom_table_sort_df.head(10)

anime_id
231      0.526448
25157    0.486146
6811     0.483627
4938     0.483627
34055    0.483627
249      0.483627
12503    0.458438
28285    0.443325
32379    0.440806
177      0.440806
dtype: float64

In [85]:
anime_df.loc[anime_df['anime_id'].isin(recom_table_sort_df.head(10).keys())]


,anime_id,name,genre
173,6811,InuYasha: Kanketsu-hen,"[Action, Adventure, Comedy, Demons, Fantas..."
286,4938,Tsubasa: Shunraiki,"[Action, Adventure, Drama, Fantasy, Magic,..."
690,249,InuYasha,"[Action, Adventure, Comedy, Demons, Fantas..."
1129,177,Tsubasa Chronicle,"[Action, Adventure, Fantasy, Magic, Romanc..."
1480,28285,Trinity Seven OVA,"[Action, Comedy, Ecchi, Fantasy, Harem, M..."
1845,25157,Trinity Seven,"[Action, Comedy, Ecchi, Fantasy, Harem, M..."
4052,32379,Berserk (2016),"[Action, Adventure, Demons, Drama, Fantasy..."
4164,12503,Mirai Nikki (TV): Ura Mirai Nikki,"[Action, Comedy, Mystery, Psychological, S..."
5917,231,Asagiri no Miko,"[Action, Comedy, Drama, Fantasy, Magic, S..."
10924,34055,Berserk (2017),"[Action, Adventure, Demons, Drama, Fantasy..."
